# **Bitcoin price prediction with PySpark - Data crawling**
## Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author: Corsi Danilo (1742375) - corsi.1742375@studenti.uniroma1.it



---


Description: data crawling on Bitcoin blochckain by querying Blockchain.com website.

# Global constants, dependencies, libraries and tools

In [1]:
# Main constants
GDRIVE_DIR = "/content/drive"

In [2]:
# Datasets dir
GDRIVE_DATASET_RAW_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/raw"

# Datasets name
DATASET_NAME = "bitcoin_blockchain_data_30min"

# Datasets path
GDRIVE_DATASET_RAW = GDRIVE_DATASET_RAW_DIR + "/" + DATASET_NAME + ".parquet"

In [3]:
# Useful imports
import pandas as pd
import functools

from google.colab import drive

from datetime import date

In [4]:
# Point Colaboratory to Google Drive
from google.colab import drive

# Define GDrive paths
drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


# Metrics and parameters
I chose to collect data on the Bitcoin blockchain using the API of the website Blockchain.org, the most relevant information was retrieved from the year 2016 to the present day (a period for which there were moments of high volatility but also a lot of price lateralization).

The features taken under consideration were divided into several categories:

**Currency Statistics**

- **market-price:** the average USD market price across major bitcoin exchanges.
- **trade-volume:** the total USD value of trading volume on major bitcoin exchanges.
- **total-bitcoins:** the total number of mined bitcoin that are currently circulating on the network.
- **market-cap:** the total USD value of bitcoin in circulation.

**Block Details**

- **blocks-size:** the total size of the blockchain minus database indexes in megabytes.
- **avg-block-size:** the average block size over the past 24 hours in megabytes.
- **n-transactions-total:** the total number of transactions on the blockchain.
- **n-transactions-per-block:** the average number of transactions per block over the past 24 hours.

**Mining Information**

- **hash-rate:** the estimated number of terahashes per second the bitcoin network is performing in the last 24 hours.
- **difficulty:** a relative measure of how difficult it is to mine a new block for the blockchain.
- **miners-revenue:** total value of coinbase block rewards and transaction fees paid to miners.
- **transaction-fees-usd:** the total USD value of all transaction fees paid to miners. This does not include coinbase block rewards.

**Network Activity**

- **n-unique-addresses:** the total number of unique addresses used on the blockchain.
- **n-transactions:** the total number of confirmed transactions per day.
- **estimated-transaction-volume-usd:** the total estimated value in USD of transactions on the blockchain.

In [25]:
# Define the parameters
timespan = "6years" # Duration of the data (it was necessary to define it since it is possible to make requests for up to 6 years)
start_date = "2016-01-01"
continue_date = "2021-12-31" # 6 years from start_date
end_date = str(date.today())

# Metrics considered
metrics = [
          # Currency Statistics
          "market-price",
          "trade-volume",

          # Block Details
          "blocks-size",
          "avg-block-size",
          "n-transactions-total",
          "n-transactions-per-block",

          # Mining Information
          "hash-rate",
          "difficulty",
          "miners-revenue",
          "transaction-fees-usd",

          # Network Activity
          "n-unique-addresses",
          "n-transactions",
          "estimated-transaction-volume-usd"
]

# Data crawling

In [26]:
def data_crawler(timespan, metrics, start_date, continue_date, end_date):
    # API info
    url1 = f'https://api.blockchain.info/charts/{metrics}?timespan={timespan}&start={start_date}&format=csv'
    url2 = f'https://api.blockchain.info/charts/{metrics}?timespan={timespan}&start={continue_date}&format=csv'

    # Obtain data
    data1 = pd.read_csv(url1, names=['timestamp', metrics])
    data2 = pd.read_csv(url2, names=['timestamp', metrics])

    # Concat by rows
    all_data = pd.concat([data1, data2])

    # Transform "timestamp" to datetime type
    all_data['timestamp'] = pd.to_datetime(all_data["timestamp"])

    # Select data up to the end date
    all_data = all_data[(all_data['timestamp'] < end_date)]

    return all_data

In [27]:
# Merge the data
merge = functools.partial(pd.merge, on='timestamp')

# Gain blockchain bata from Blockchain.com API
df1 = functools.reduce(merge, [data_crawler(timespan, metric, start_date, continue_date, end_date) for metric in metrics])
df1

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,transactions-per-second,estimated-transaction-volume-usd
0,2016-06-11,577.95,1.999141e+06,72452.200786,0.696121,134968126,1366.403846,1.520418e+06,1.960614e+11,2.349957e+06,2.737374e+04,375798.0,213159.0,1.983333,1.622448e+08
1,2016-08-15,567.34,2.730948e+06,80068.237355,0.809770,149293157,1389.698630,1.577646e+06,2.034839e+11,1.075739e+06,3.879417e+04,354911.0,202896.0,1.733333,1.211514e+08
2,2017-06-13,2656.97,8.378747e+07,120721.302140,0.977148,231382646,2069.333333,4.757530e+06,6.787601e+11,6.081319e+06,1.300252e+06,611163.0,291776.0,2.233333,8.389207e+08
3,2020-04-18,7034.89,1.148953e+08,272696.584468,1.150726,521770657,1829.962025,1.155766e+08,1.471521e+13,1.426197e+07,1.378613e+05,562996.0,289134.0,2.666667,5.198332e+08
4,2021-12-31,47132.96,2.668890e+08,382992.583467,1.394028,699106307,1808.993289,1.797810e+08,2.427233e+13,4.474437e+07,8.518002e+05,703405.0,269540.0,2.533333,8.132766e+09
5,2022-01-13,43926.00,3.060790e+08,385136.326251,0.950327,702312053,1533.603352,2.168643e+08,2.437187e+13,4.960069e+07,4.584813e+05,695923.0,274515.0,2.566667,4.128979e+09
6,2022-01-26,36948.81,2.889578e+08,387310.781397,1.240494,705634659,2008.230769,1.721772e+08,2.664319e+13,3.135337e+07,4.544188e+05,648504.0,261070.0,2.416667,4.724487e+09
7,2022-02-08,43834.02,4.030174e+08,389558.418856,1.269142,708921648,1797.727273,2.043262e+08,2.669053e+13,4.342519e+07,6.425412e+05,685310.0,276850.0,3.000000,5.859755e+09
8,2022-02-21,38382.32,1.907498e+08,391910.254910,1.303795,712147783,1930.097561,1.710013e+08,2.796715e+13,3.002658e+07,3.673125e+05,615656.0,237402.0,2.066667,4.035714e+09
9,2022-03-06,39399.58,1.182941e+08,394137.759364,1.067880,715437707,1581.038168,1.794090e+08,2.755033e+13,3.234770e+07,3.693664e+05,555454.0,207116.0,2.200000,1.221416e+09


In [ ]:
# Check duplicated rows
len(df1['timestamp'].unique())

2790

In [ ]:
# Retrieving market capitalization and total circulating data
metrics = [
          # Currency Statistics
          "total-bitcoins",                      # Total Circulating Bitcoin: The total number of mined bitcoin that are currently circulating on the network.
          "market-cap",                          # Market Capitalization (USD): The total USD value of bitcoin in circulation.
  ]

df2 = functools.reduce(merge, [data_crawler(timespan, metric, start_date, continue_date, end_date) for metric in metrics])
df2

,timestamp,total-bitcoins,market-cap
0,2016-01-01 00:03:21,15029575.00,6.474140e+09
1,2016-01-02 15:54:15,15035125.00,6.499685e+09
2,2016-01-04 02:26:37,15040650.00,6.458253e+09
3,2016-01-05 09:50:06,15046150.00,6.489359e+09
4,2016-01-06 21:03:44,15051750.00,6.472554e+09
...,...,...,...
3000,2023-08-26 12:22:59,19468325.00,5.075392e+11
3001,2023-08-27 00:51:51,19468693.75,5.070232e+11
3002,2023-08-27 12:48:12,19469062.50,5.077921e+11
3003,2023-08-28 00:48:26,19469431.25,5.075291e+11


In [ ]:
# Check duplicated rows
len(df2['timestamp'].unique())

3005

In [ ]:
# Wipe off the timestamp's h:m:s.
df2['timestamp'] = pd.to_datetime(df2["timestamp"]).dt.normalize()

# Drop the duplicates in column "timestamp", keep the last value
df2.drop_duplicates(subset="timestamp", keep="last", inplace=True)
df2

,timestamp,total-bitcoins,market-cap
0,2016-01-01,15029575.00,6.474140e+09
1,2016-01-02,15035125.00,6.499685e+09
2,2016-01-04,15040650.00,6.458253e+09
3,2016-01-05,15046150.00,6.489359e+09
4,2016-01-06,15051750.00,6.472554e+09
...,...,...,...
2996,2023-08-24,19466843.75,5.066246e+11
2998,2023-08-25,19467581.25,5.061766e+11
3000,2023-08-26,19468325.00,5.075392e+11
3002,2023-08-27,19469062.50,5.077921e+11


In [ ]:
# Check duplicated rows
len(df2['timestamp'].unique())

2110

In [ ]:
# Add the market capitalization and total circulating data to the main dataset
all_data = pd.merge(df1, df2, how="inner", on='timestamp')
all_data = all_data.interpolate(method='ffill')
all_data

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap
0,2016-01-01,430.89,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07,15029575.00,6.474140e+09
1,2016-01-02,434.75,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07,15035125.00,6.499685e+09
2,2016-01-04,430.78,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07,15040650.00,6.458253e+09
3,2016-01-05,434.17,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08,15046150.00,6.489359e+09
4,2016-01-06,432.43,1.677504e+06,55036.950659,0.662380,101934175,1220.496454,7.022931e+05,1.038803e+11,1.702648e+06,14113.278937,312004.0,172090.0,1.007001e+08,15051750.00,6.472554e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,2023-08-24,26426.74,1.477820e+08,505878.748122,1.570589,883712238,2871.589928,3.843290e+08,5.562144e+13,2.333327e+07,459297.742939,660102.0,399151.0,2.313068e+09,19466843.75,5.066246e+11
2102,2023-08-25,26165.40,8.886319e+07,506096.718886,1.627875,884113026,3149.570370,3.732692e+08,5.562144e+13,2.246792e+07,470208.975992,706946.0,425192.0,2.858718e+09,19467581.25,5.061766e+11
2103,2023-08-26,26049.42,9.463832e+07,506316.642688,1.663670,884538734,3631.481752,3.787991e+08,5.562144e+13,2.307548e+07,424553.398959,747932.0,497513.0,1.073304e+09,19468325.00,5.075392e+11
2104,2023-08-27,26006.44,1.722276e+07,506544.494419,1.694264,885036076,3565.465517,3.207350e+08,5.562144e+13,1.927180e+07,351591.993834,606069.0,413594.0,1.002188e+09,19469062.50,5.077921e+11


In [ ]:
# Check nan values
all_data[all_data.isnull().T.any()]

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap


In [ ]:
# Check duplicated rows
len(all_data['timestamp'].unique())

2106

In [ ]:
all_data

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap
0,2016-01-01,430.89,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07,15029575.00,6.474140e+09
1,2016-01-02,434.75,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07,15035125.00,6.499685e+09
2,2016-01-04,430.78,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07,15040650.00,6.458253e+09
3,2016-01-05,434.17,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08,15046150.00,6.489359e+09
4,2016-01-06,432.43,1.677504e+06,55036.950659,0.662380,101934175,1220.496454,7.022931e+05,1.038803e+11,1.702648e+06,14113.278937,312004.0,172090.0,1.007001e+08,15051750.00,6.472554e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,2023-08-24,26426.74,1.477820e+08,505878.748122,1.570589,883712238,2871.589928,3.843290e+08,5.562144e+13,2.333327e+07,459297.742939,660102.0,399151.0,2.313068e+09,19466843.75,5.066246e+11
2102,2023-08-25,26165.40,8.886319e+07,506096.718886,1.627875,884113026,3149.570370,3.732692e+08,5.562144e+13,2.246792e+07,470208.975992,706946.0,425192.0,2.858718e+09,19467581.25,5.061766e+11
2103,2023-08-26,26049.42,9.463832e+07,506316.642688,1.663670,884538734,3631.481752,3.787991e+08,5.562144e+13,2.307548e+07,424553.398959,747932.0,497513.0,1.073304e+09,19468325.00,5.075392e+11
2104,2023-08-27,26006.44,1.722276e+07,506544.494419,1.694264,885036076,3565.465517,3.207350e+08,5.562144e+13,1.927180e+07,351591.993834,606069.0,413594.0,1.002188e+09,19469062.50,5.077921e+11


In [ ]:
# Reorder colunmns
new_columns = ['timestamp', 'market-price', 'total-bitcoins', 'market-cap'] + [col for col in all_data.columns if col not in ['timestamp', 'market-price', 'total-bitcoins', 'market-cap']]
all_data = all_data.reindex(columns=new_columns)
all_data

,timestamp,market-price,total-bitcoins,market-cap,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
0,2016-01-01,430.89,15029575.00,6.474140e+09,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07
1,2016-01-02,434.75,15035125.00,6.499685e+09,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07
2,2016-01-04,430.78,15040650.00,6.458253e+09,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07
3,2016-01-05,434.17,15046150.00,6.489359e+09,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08
4,2016-01-06,432.43,15051750.00,6.472554e+09,1.677504e+06,55036.950659,0.662380,101934175,1220.496454,7.022931e+05,1.038803e+11,1.702648e+06,14113.278937,312004.0,172090.0,1.007001e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,2023-08-24,26426.74,19466843.75,5.066246e+11,1.477820e+08,505878.748122,1.570589,883712238,2871.589928,3.843290e+08,5.562144e+13,2.333327e+07,459297.742939,660102.0,399151.0,2.313068e+09
2102,2023-08-25,26165.40,19467581.25,5.061766e+11,8.886319e+07,506096.718886,1.627875,884113026,3149.570370,3.732692e+08,5.562144e+13,2.246792e+07,470208.975992,706946.0,425192.0,2.858718e+09
2103,2023-08-26,26049.42,19468325.00,5.075392e+11,9.463832e+07,506316.642688,1.663670,884538734,3631.481752,3.787991e+08,5.562144e+13,2.307548e+07,424553.398959,747932.0,497513.0,1.073304e+09
2104,2023-08-27,26006.44,19469062.50,5.077921e+11,1.722276e+07,506544.494419,1.694264,885036076,3565.465517,3.207350e+08,5.562144e+13,1.927180e+07,351591.993834,606069.0,413594.0,1.002188e+09



Once we have the daily dataset we will sample it at a frequency of 30 minutes (30T) using the resample method.

This means that the data will be organized in 30-minute time-frame, and an interpolation method will be used to fill in any missing data or holes in the dataset by estimating missing values based on the surrounding known values.

In [ ]:
# Upsampling to 30min by interpolate
all_data.set_index('timestamp', inplace=True)
all_data_30m = all_data.resample('30T').interpolate()
all_data_30m

,market-price,total-bitcoins,market-cap,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
timestamp,,,,,,,,,,,,,,,
2016-01-01 00:00:00,430.890000,1.502958e+07,6.474140e+09,2.860153e+06,54604.791735,0.493407,1.011557e+08,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.000000,124092.000000,5.834626e+07
2016-01-01 00:30:00,430.970417,1.502969e+07,6.474672e+09,2.834859e+06,54606.164986,0.495206,1.011583e+08,921.463506,6.973443e+05,1.038803e+11,1.557199e+06,8887.551433,264578.937500,124611.708333,5.818972e+07
2016-01-01 01:00:00,431.050833,1.502981e+07,6.475204e+09,2.809565e+06,54607.538237,0.497004,1.011608e+08,923.727011,6.975595e+05,1.038803e+11,1.559629e+06,8992.039015,266036.875000,125131.416667,5.803318e+07
2016-01-01 01:30:00,431.131250,1.502992e+07,6.475736e+09,2.784271e+06,54608.911488,0.498803,1.011634e+08,925.990517,6.977747e+05,1.038803e+11,1.562059e+06,9096.526597,267494.812500,125651.125000,5.787664e+07
2016-01-01 02:00:00,431.211667,1.503004e+07,6.476268e+09,2.758977e+06,54610.284739,0.500602,1.011659e+08,928.254023,6.979898e+05,1.038803e+11,1.564490e+06,9201.014178,268952.750000,126170.833333,5.772010e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-27 22:00:00,26081.918333,1.946974e+07,5.088092e+11,2.407792e+07,506724.733752,1.592551,8.854152e+08,2871.129895,3.258041e+08,5.562144e+13,1.963350e+07,434927.724200,634362.833333,338199.083333,2.142855e+09
2023-08-27 22:30:00,26083.633750,1.946975e+07,5.088323e+11,2.423372e+07,506728.830101,1.590239,8.854239e+08,2855.349540,3.259193e+08,5.562144e+13,1.964172e+07,436821.718072,635005.875000,336485.562500,2.168779e+09
2023-08-27 23:00:00,26085.349167,1.946977e+07,5.088554e+11,2.438952e+07,506732.926449,1.587928,8.854325e+08,2839.569185,3.260345e+08,5.562144e+13,1.964994e+07,438715.711944,635648.916667,334772.041667,2.194703e+09


# Saving dataset

In [ ]:
# Save the 30m dataset
all_data_30m.to_parquet(GDRIVE_DATASET_RAW)